In [1]:
import os
from tqdm import tqdm       # for showing progress when training
import pandas as pd         # for load data
import numpy as np
import open3d as o3d        # for getting point cloud register
# import math as m            # for coordinate conversion
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
from einops import rearrange, repeat
#test update for branch wild

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### Preperation (NOTE: Using meter as unit)

In [2]:
# Utility functions
# convert pointcloud from cartisean coordinate to spherical coordinate
def cart2sph(xyz):
    x = xyz[:,0]
    y = xyz[:,1]
    z = xyz[:,2]
    XsqPlusYsq = x**2 + y**2
    r = np.sqrt(list(XsqPlusYsq + z**2))
    elev = np.arctan2(list(z), np.sqrt(list(XsqPlusYsq)))
    pan = np.arctan2(list(x), list(y))
    output = np.array([r, elev, pan])
    return rearrange(output, 'a b -> b a') #take transpose

def sph2cart(ang):
    ele = ang[:,0]
    pan = ang[:,1]
    x = np.cos(ele)*np.cos(pan)
    y = np.cos(ele)*np.sin(pan)
    z = np.sin(ele)
    output = np.array([x,y,z])
    return rearrange(output, 'a b -> b a') #take transpose

In [3]:
def loadData():
    # Specify the directory path
    dataset_path = 'datasets/testing1'

    # List all files in the specified path, ignoring directories
    files = [f for f in os.listdir(dataset_path) if os.path.isfile(os.path.join(dataset_path, f))]
    files.sort()

    # read the files
    points_xyz = []
    for s in files:
        path = 'datasets/testing1/' + s
        df = pd.read_csv(path)
        a = df.to_numpy()
        points_xyz.append(a[:,8:11])
    return points_xyz

def getTransformation(data):
    """ 
    Accept input of list of numpy array of n*3 size, 
    return list of 4*4 numpy array of transformation matrix
    each are transformation needed from each frame to point cloud in fame 0
    """
    # convert numpy array 
    point_clouds = []
    for d in data:
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(d)
        point_clouds.append(pcd)
    threshold = 2.0 
    trans_init = np.eye(4)  # Initial transformation matrix
    transformations = []

    # the following may take a while
    for i in range(len(point_clouds)):
        source_pcd = point_clouds[i]
        target_pcd = point_clouds[0]
        reg_p2p = o3d.pipelines.registration.registration_icp(
                    source_pcd, 
                    target_pcd, 
                    threshold, 
                    trans_init,
                    o3d.pipelines.registration.TransformationEstimationPointToPoint())
        transformations.append(reg_p2p.transformation)
    return transformations
    
def prepareData(points_xyz):
    # get transformations
    transformations = getTransformation(points_xyz)

    # register all points onto same global coordinte (global coordinate align with frame 0 coordinate)
    points_reg_xyz = []
    for i, points in enumerate(points_xyz):
        ones = np.ones((len(points), 1))
        homo_points = np.hstack((points, ones))
        # apply transformation
        t = transformations[i]
        t = rearrange(t, 'a b -> b a')
        reg_points = homo_points@t
        reg_points = reg_points / rearrange(reg_points[:,3], 'a -> a 1')
        reg_points = reg_points[:,0:3]
        points_reg_xyz.append(reg_points)

    # create a list of origins 
    centres = [(t@np.array([[0],[0],[0],[1]]))[0:3,0] for t in transformations]
    
    # get the angular direction and distance of rays    
    points_sph = []
    for i, points in enumerate(points_reg_xyz):
        relative_loc = points - centres[i]
        points_sph.append(cart2sph(relative_loc))

    # tile sensor centre 
    centres_tiled = []
    for i, centre in enumerate(centres):
        l = len(points_sph[i])
        temp = np.tile(centre, (l, 1))
        centres_tiled.append(temp)
    
    # stack everything into a matrix of size n * 6
    # where n is number of points, 6 corrsponds to:
    # distance, elevation, pan, x of camera, y of camera, z of camera
    # stack the points into one big matrix
    stacked = []
    for i in range(len(points_sph)):
        temp = np.hstack((points_sph[i], centres_tiled[i]))
        stacked.append(temp)

    dataset = np.array([])
    for i in range(len(stacked)):
        if i == 0:
            dataset = stacked[i]
        else:
            dataset = np.vstack((dataset, stacked[i]))

    # Mid pass filter, for distance value between 2 and 50 meter
    mask1 = dataset[:,0] > 2
    dataset = dataset[mask1]
    mask2 = dataset[:,0] < 50
    dataset = dataset[mask2]
    np.random.shuffle(dataset)      # shuffle for good practice

    return dataset

In [4]:
class LiDAR_NeRF(nn.Module):
    def __init__(self, embedding_dim_pos = 10, embedding_dim_dir = 4, hidden_dim = 256, device = 'cuda'):
        super(LiDAR_NeRF, self).__init__()
        self.device = device
        self.embedding_dim_dir = embedding_dim_dir
        self.embedding_dim_pos = embedding_dim_pos
        self.block1 = nn.Sequential(
            nn.Linear(embedding_dim_pos * 6 + 3 + embedding_dim_dir * 4 + 2, hidden_dim), nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim), nn.ReLU(),               
            nn.Linear(hidden_dim, hidden_dim), nn.ReLU(),               
            nn.Linear(hidden_dim, hidden_dim), nn.ReLU(),               
        )
        
        self.block2 = nn.Sequential(
            nn.Linear(embedding_dim_pos * 6 + 3 + embedding_dim_dir * 4 + 2 + hidden_dim, hidden_dim), nn.ReLU(),               
            nn.Linear(hidden_dim, hidden_dim), nn.ReLU(),               
            nn.Linear(hidden_dim, hidden_dim), nn.ReLU(),               
            nn.Linear(hidden_dim, hidden_dim), nn.ReLU(),
            nn.Linear(hidden_dim,1)
        )
        
    @staticmethod
    def positional_encoding(x, L):
        out = [x]
        for j in range(L):
            out.append(torch.sin(2 ** j * x))
            out.append(torch.cos(2 ** j * x))
        return torch.cat(out, dim=1)

    def forward(self, o, d):
        emb_x = self.positional_encoding(o, self.embedding_dim_pos)
        emb_d = self.positional_encoding(d, self.embedding_dim_dir)
        input = torch.hstack((emb_x,emb_d)).to(dtype=torch.float32)
        temp = self.block1(input)
        input2 = torch.hstack((temp, input)).to(dtype=torch.float32) # add skip input
        output = self.block2(input2)
        return output

In [5]:
def getSamples(origins, angles, ground_truth_distance, num_bins = 100):
    elev = angles[:,0]
    pan = angles[:,1]
    dir_x = torch.tensor(np.cos(elev)*np.cos(pan))      # [batch_size]
    dir_y = torch.tensor(np.cos(elev)*np.sin(pan))      # [batch_size]
    dir_z = torch.tensor(np.sin(elev))
    gt_tensor = torch.tensor(ground_truth_distance)

    # create a list of magnitudes with even spacing from 0 to 1
    t = torch.linspace(0,1, num_bins).expand(dir_x.shape[0], num_bins)  # [batch_size, num_bins]
    
    # preterb the spacing
    mid = (t[:, :-1] + t[:, 1:]) / 2.
    lower = torch.cat((t[:, :1], mid), -1)
    upper = torch.cat((mid, t[:, -1:]), -1)
    u = torch.rand(t.shape)
    t = lower + (upper - lower) * u  # [batch_size, nb_bins]
    t = rearrange(t, 'a b -> b a')  # [num_bins, batch_size]  take transpose so that multiplication can broadcast
    
    # multiply the magnitude to ground truth distance and add 3 meter
    t = torch.sqrt(t)
    t = gt_tensor*t
    t += 5

    # convert magnitudes into positions by multiplying it to the unit vector
    pos_x = dir_x*t     # [num_bins, batch_size]
    pos_y = dir_y*t
    pos_z = dir_z*t
    # concat them for output
    multiplied = rearrange([pos_x,pos_y,pos_z], 'c b n  -> (n b) c')   # [num_bin*batchsize, 3]
    # tile the origin values
    origins_tiled = repeat(origins, 'n c -> (n b) c', b = num_bins) # [num_bin*batch_size, 3]
    pos = torch.tensor(origins_tiled) + multiplied
    # tile the angle too
    angles_tiled = torch.tensor(repeat(angles, 'n c -> (n b) c', b = num_bins))
    return pos, angles_tiled, origins_tiled



In [6]:
def getUpSamples(origins, angles, gt_distance, num_rolls = 1):
    upsample_pos = torch.empty(0,3)
    upsample_ang = torch.empty(0,2)
    upsample_gt_dist = torch.empty(0,1)

    for num_roll in range(1, num_rolls+1):
        # first, we prepare pairs of data, where one of them has a shorter ray, and another has a longer ray
        gt_distance_rolled = torch.roll(gt_distance, num_roll, 0)
        condition =  gt_distance < gt_distance_rolled
        condition = rearrange(condition, 'a -> a 1')

        dir = torch.tensor(sph2cart(angles))
        gt_dist = rearrange(gt_distance, 'a -> a 1')
        gt_distance_rolled = rearrange(gt_distance_rolled, 'a -> a 1')
        pos = gt_dist * dir

        pos_shorter = torch.where(condition, pos, torch.roll(pos, num_roll, 0))
        origins_shorter = torch.where(condition, origins, torch.roll(origins, num_roll, 0))
        angles_shorter = torch.where(condition, angles, torch.roll(angles, num_roll, 0))
        gt_dist_shorter = torch.where(condition, gt_dist, gt_distance_rolled)
        pos_longer = torch.where(condition, torch.roll(pos, num_roll, 0), pos)
        origins_longer = torch.where(condition, torch.roll(origins, num_roll, 0), origins)
        angles_longer = torch.where(condition, torch.roll(angles, num_roll, 0), angles)
        gt_dist_longer = torch.where(condition, gt_distance_rolled, gt_dist)
        
        # check if angle between pairs are small
        diff = torch.abs(angles_shorter - angles_longer)
        mask_small_ang = (diff < 0.09).all(dim=1)   ### NOTE: hardcoded 0.09 radient difference max

        # check if origin between pairs are small
        diff2 = torch.abs(origins_shorter - origins_longer)
        mask_small_org = (diff2 < 0.2).all(dim=1)   ### pass if coordinate in origin are less than 20cm in all dimensions
        
        # # check if the target location is close
        # diff3 = torch.abs(pos_shorter - pos_longer)
        # mask_small_dis = (diff3 < 0.2).all(dim=1)

        # get masks for both cases
        mask_same_org = mask_small_ang & mask_small_org
        # mask_same_tag = mask_small_ang & mask_small_dis

        ### Handling case of same origin
        # prepare set where rays are to be upsampled 
        # ensuring samples are at rays that are longer
        angles_from_same_org = angles_longer[mask_same_org]
        origins_from_same_org = origins_longer[mask_same_org]
        gt_dist_to_same_org = gt_dist_shorter[mask_same_org] 
        pos_to_same_org = pos_shorter[mask_same_org]

        if angles_from_same_org.shape[0] == 0:
            continue   # skip if there are no points available for upsampling

        # calculate upsampling position
        num_bins = 20 
        t = torch.linspace(0,1, num_bins).expand(angles_from_same_org.shape[0], num_bins)  # [batch_size, num_bins]
        
        # preterb the spacing
        mid = (t[:, :-1] + t[:, 1:]) / 2.
        lower = torch.cat((t[:, :1], mid), -1)
        upper = torch.cat((mid, t[:, -1:]), -1)
        u = torch.rand(t.shape)
        t = lower + (upper - lower) * u  # [batch_size, nb_bins]
        t = rearrange(t, 'a b -> b a')  # [num_bins, batch_size]  take transpose so that multiplication can broadcast
        t = rearrange(t, 'a b -> (b a) 1')
        t = torch.sqrt(t)
        
        # get the sampling positions
        origins_from_tiled = repeat(origins_from_same_org, 'n c -> (n b) c', b = num_bins)
        dir_from = torch.tensor(sph2cart(angles_from_same_org))
        dir_from_tiled = repeat(dir_from, 'n c -> (n b) c', b = num_bins)
        gt_dist_to_tiled = repeat(gt_dist_to_same_org, 'n c -> (n b) c', b = num_bins)     

        pos_from = origins_from_tiled + dir_from_tiled * t * gt_dist_to_tiled
        pos_to_tiled = repeat(pos_to_same_org, 'n c -> (n b) c', b = num_bins)

        # also calculte the ground truth distance of our up sampled location
        sample_sph = cart2sph(pos_from - pos_to_tiled)
        sample_direction = torch.tensor(sample_sph[:,1:])
        sample_gt_distance = torch.tensor(sample_sph[:,0])

        # add one more dimension to sample_gt_distance
        sample_gt_distance = rearrange(sample_gt_distance, 'a -> a 1')

        upsample_pos = torch.vstack((upsample_pos, pos_from))
        upsample_ang = torch.vstack((upsample_ang, sample_direction))
        upsample_gt_dist = torch.vstack((upsample_gt_dist, sample_gt_distance))

    # return pos_from , torch.tensor(sample_direction), torch.tensor(sample_gt_distance)
    return upsample_pos, upsample_ang, upsample_gt_dist

In [7]:
# returns pytorch tensor of sigmoid of projected SDF
def getTargetValues(sample_positions, gt_distance, origins, num_bins=100):
    # calculate distance from sample_position
    temp = torch.tensor((sample_positions - origins)**2)
    pos_distance = torch.sqrt(torch.sum(temp, dim=1, keepdim=True))
    
    # find the "projected" value
    sigmoid = nn.Sigmoid()
    values = sigmoid(-(pos_distance - gt_distance))
    return values

In [8]:
# # constants
# num_bins = 100
# device = 'cuda'

# # sample data for testing
# points = loadData()
# dataset = prepareData(points)
# test_batch = torch.tensor(dataset[0:256,:])
# gt_distance = test_batch[:,0]
# angles = test_batch[:,1:3]
# origins = test_batch[:,3:6]
# upsample_pos, upsample_ang, upsample_gt_distance = getUpSamples(origins, angles,  gt_distance, num_rolls=5)


# sample_pos, sample_ang, sample_org = getSamples(origins, angles, gt_distance, num_bins = num_bins)
# gt_distance_tiled = repeat(gt_distance, 'b -> (b n) 1', n = num_bins)

# # stack the upsampled position to sampled positions
# pos = (torch.vstack((sample_pos, upsample_pos))).to(device)
# ang = (torch.vstack((sample_ang, upsample_ang))).to(device)
# gt_dis = (torch.vstack((gt_distance_tiled,upsample_gt_distance) )).to(device)
# org = (torch.vstack((sample_org, upsample_pos))).to(device)

# # inference and training
# model = LiDAR_NeRF(hidden_dim=512).to(device)
# rendered_value = model(pos, ang)
# sigmoid = nn.Sigmoid()
# rendered_value_sigmoided = sigmoid(rendered_value)
# actual_value_sigmoided = (getTargetValues(pos, gt_dis, org)).to(dtype = torch.float32)
# # loss = lossBCE(rendered_value, actual_value_sigmoided)  # + lossEikonal(model)
# loss_bce = nn.BCELoss()
# loss = loss_bce(rendered_value_sigmoided, actual_value_sigmoided)


In [42]:
def train(model, optimizer, scheduler, dataloader, device = 'cuda', epoch = int(1e5), num_bins = 100):
    training_losses = []
    for _ in tqdm(range(epoch)):
        for batch in dataloader:
            # parse the batch
            ground_truth_distance = batch[:,0]
            angles = batch[:,1:3]
            origins = batch[:,3:6]
            upsample_pos, upsample_ang, upsample_gt_distance = getUpSamples(origins, angles, ground_truth_distance, num_rolls=0)
            sample_pos, sample_ang, sample_org = getSamples(origins, angles, ground_truth_distance, num_bins=num_bins)
            # tile distances
            gt_distance_tiled = repeat(ground_truth_distance, 'b -> (b n) 1', n=num_bins)

            # stack the upsampled position to sampled positions
            pos = (torch.vstack((sample_pos, upsample_pos))).to(device)
            ang = (torch.vstack((sample_ang, upsample_ang))).to(device)
            gt_dis = (torch.vstack((gt_distance_tiled,upsample_gt_distance))).to(device)
            org = (torch.vstack((sample_org, upsample_pos))).to(device)
            
            # inference
            rendered_value = model(pos, ang)
            sigmoid = nn.Sigmoid()
            rendered_value_sigmoid = sigmoid(rendered_value)
            actual_value_sigmoided = (getTargetValues(pos, gt_dis, org)).to(dtype = torch.float32)

            # loss = lossBCE(rendered_value, actual_value_sigmoided)  # + lossEikonal(model)
            # back propergate
            loss_bce = nn.BCELoss()
            loss = loss_bce(rendered_value_sigmoid, actual_value_sigmoided)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            training_losses.append(loss.item())
        scheduler.step()
        print(loss.item())
    return training_losses
    

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {device} device")
points = loadData()
print("loaded data")
data_matrix = prepareData(points)
print("prepared data")
training_dataset = torch.from_numpy(data_matrix)
data_loader = DataLoader(training_dataset, batch_size=512, shuffle = True)
model = LiDAR_NeRF(hidden_dim=512, embedding_dim_dir=15, device = device).to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=5e-4)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[2, 4, 8, 16], gamma=0.5)
losses = train(model, optimizer, scheduler, data_loader, epoch = 8, device=device)

Using cuda device
loaded data
prepared data


  0%|          | 0/8 [00:00<?, ?it/s]/tmp/ipykernel_32038/1749373479.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dir_x = torch.tensor(np.cos(elev)*np.cos(pan))      # [batch_size]
/tmp/ipykernel_32038/1749373479.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dir_y = torch.tensor(np.cos(elev)*np.sin(pan))      # [batch_size]
/tmp/ipykernel_32038/1749373479.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dir_z = torch.tensor(np.sin(elev))
/tmp/ipykernel_32038/1749373479.py:7: UserWarning: To copy construct from a tensor, it

0.2858654260635376


 25%|██▌       | 2/8 [11:00<32:57, 329.62s/it]

0.28038448095321655


...

0.2841636538505554


 88%|████████▊ | 7/8 [38:11<05:26, 326.38s/it]

0.26745888590812683


100%|██████████| 8/8 [43:34<00:00, 326.85s/it]

0.2780597507953644


In [15]:
### Save the model
torch.save(model.state_dict(), 'local/models/version2_trial4.pth')

In [32]:
#### Load the model and try to "visualize" the model's datapoints
model_evel = LiDAR_NeRF(hidden_dim=512, embedding_dim_dir=15, device = 'cpu')
model_evel.load_state_dict(torch.load('local/models/version2_trial4.pth'))
model_evel.eval(); # Set the model to inference mode

In [13]:
### Render some structured pointcloud for evaluation
with torch.no_grad():
    dist = 0.1 # initial distanc forvisualization
    pos = torch.zeros((100000,3))
    ele = torch.linspace(-0.34, 0.3, 100)
    pan = torch.linspace(-3.14, 3.14, 1000)
    ele_tiled = repeat(ele, 'n -> (r n) 1', r = 1000)
    pan_tiled = repeat(pan, 'n -> (n r) 1', r = 100)
    ang = torch.cat((ele_tiled, pan_tiled), dim=1)

    # direction for each "point" from camera centre
    directions = torch.tensor(sph2cart(np.array(ang)))

    for i in range(500):
        output2 = model_evel(pos, ang)
        temp = torch.sign(output2)
        pos += directions * dist * temp
        # dist /= 2


In [40]:
### Render some structured pointcloud for evaluation
with torch.no_grad():
    dist = 32 # initial distanc forvisualization
    pos = torch.zeros((100000,3))
    pos[:,1] += 2
    ele = torch.linspace(-0.34, 0.3, 100)
    pan = torch.linspace(-3.14, 3.14, 1000)
    ele_tiled = repeat(ele, 'n -> (r n) 1', r = 1000)
    pan_tiled = repeat(pan, 'n -> (n r) 1', r = 100)
    ang = torch.cat((ele_tiled, pan_tiled), dim=1)

    # direction for each "point" from camera centre
    directions = torch.tensor(sph2cart(np.array(ang)))

    for i in range(10):
        output2 = model_evel(pos, ang)
        temp = torch.sign(output2)
        pos += directions * dist * temp
        dist /= 2


In [41]:
### Save to csv for visualization
df_temp = pd.read_csv('local/visualize/dummy.csv')
df_temp = df_temp.head(100000)
pos_np = pos.numpy()
df_temp['X'] = pos_np[:,0]
df_temp['Y'] = pos_np[:,1]
df_temp['Z'] = pos_np[:,2]
df_temp.to_csv('local/visualize/visualize.csv', index=False)